In [1]:
import os

In [2]:
import datasets

In [3]:
local_dir = '/cwStorage/nodecw_group/jijh/hest_1k'

In [ ]:
help(datasets.load_dataset)

In [5]:
import datasets

# 数据集脚本的本地路径
dataset_script_path = '/cwStorage/nodecw_group/jijh/hest_1k/hest.py'

# 本地数据集路径
data_dir = '/cwStorage/nodecw_group/jijh/hest_1k'

# 加载本地数据集脚本
dataset = datasets.load_dataset(
    dataset_script_path,
    data_dir=data_dir,
    name="custom_config"  # 使用自定义配置名称
)

# 示例：查看数据集的训练集
print(dataset['train'])

In [254]:
dataset['train'][]

In [4]:
os.chdir("/home1/jijh/diffusion_project/ADiffusion")

In [5]:
os.getcwd()

In [6]:

import importlib
from src.pipeline.hest_loading import HESTDataset



In [8]:
# 假设数据路径
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"

# 初始化数据集管理
dataset = HESTDataset(data_dir=data_dir)

In [9]:
dataset.meta_df

In [259]:
dataset.meta_df['species'].value_counts()

In [15]:
# 查询器官为 "Brain" 且疾病状态为 "Healthy" 的样本
brain_samples = dataset.get_samples(
    organ="Brain", 
    oncotree_code=None,  # 根据需要填写
    sample_ids=None,       # 根据需要填写
    species='Mus musculus'          # 根据需要填写
)

print(f"找到 {len(brain_samples)} 个样本。")
print(brain_samples[:3])  # 仅预览前三个

In [214]:
# 查询器官为 "Brain" 且疾病状态为 "Healthy" 的样本
brain_samples = dataset.get_samples(
    organ="Brain", 
    oncotree_code=None,  # 根据需要填写
    sample_ids=None       # 根据需要填写
)

print(f"找到 {len(brain_samples)} 个样本。")
print(brain_samples[:3])  # 仅预览前三个

In [261]:
# 随便选一个样本
sample = brain_samples[0]
print(sample)

In [262]:
sample.load_st_data()
sample.load_wsi()

In [263]:
sample.adata

In [264]:
sample.wsi

In [265]:
sample.spatial_plot_path

In [266]:
# --- (1) 创建综合可视化图 ---
# 默认使用预生成的空间转录组图像，如果没有则实时计算
sample.visualize_comparison(
    color='clusters',
    use_precomputed_spatial_plot=True  # 可以手动切换到 False 以实时计算
)

In [248]:
test_sample = brain_samples[1]
test_sample.load_st_data()
test_sample.load_wsi()
test_sample.visualize_comparison(
    color='clusters',
    use_precomputed_spatial_plot=True  # 可以手动切换到 False 以实时计算
)

In [267]:
dataset.compute_metrics_statistics(samples=brain_samples)

In [269]:
sample.metadata_dict

In [270]:
# --- (1) 创建综合可视化图 ---
# 默认使用预生成的空间转录组图像，如果没有则实时计算
sample.visualize_comparison(
    color='clusters',
    use_precomputed_spatial_plot=True  # 可以手动切换到 False 以实时计算
)

# Open Slide 使用

In [272]:
type(sample.wsi)

In [275]:
# Print basic metadata
slide = sample.wsi

print("Dimensions (level 0):", slide.dimensions)
print("Number of levels:", slide.level_count)
print("Dimensions per level:", slide.level_dimensions)
print("Metadata properties:")
for key, value in slide.properties.items():
    print(f"  {key}: {value}")


In [276]:
from PIL import Image

# Define the region: top-left corner at (x, y), level number, and the size (width, height)
location = (0, 0)  # top-left corner in level 0 coordinates
level = 0          # highest resolution; use a higher level for a lower resolution
size = (512, 512)  # size of the region to extract

# Read the region; the result is a PIL Image
region = slide.read_region(location, level, size)

# Display the region (this will open the image in your default image viewer)
region.show()

In [277]:
region

In [278]:
type(region)

In [282]:
desired_downsample = 16.0  # e.g., you want a quarter of the full resolution
best_level = slide.get_best_level_for_downsample(desired_downsample)
print("Best level for a downsample of 16.0 is:", best_level)

In [283]:
# Choose the desired level (e.g., level 2)
target_level = 3

# Get dimensions for the chosen level
level_dimensions = slide.level_dimensions[target_level]

# Extract the full image at the target level.
# Note: location is (0, 0) in level 0 coordinates and size is in the units of the chosen level.
region = slide.read_region((0, 0), target_level, level_dimensions)

# Convert the image from RGBA to RGB (if transparency is not needed)
region_rgb = region.convert("RGB")

# Display the image using the default image viewer (PIL's show method)
region_rgb.show()

# Always remember to close the slide after you are done
slide.close()

In [284]:
region_rgb

# 尝试切割

In [289]:
sample.adata.obsm['spatial']

In [291]:
sample.adata

In [318]:
sample.adata.obs

In [311]:
type(sample.adata.uns["spatial"])

In [317]:
plt.imshow(sample.adata.uns["spatial"]['ST']['images']['downscaled_fullres'])

In [295]:
import scanpy as sc
adata_test = sample.adata.to_memory().copy()
sc.pl.spatial(adata_test, color='n_counts', spot_size=10)

In [301]:
del adata_test.uns['spatial']

In [302]:
adata_test

In [307]:
sc.pl.spatial(adata_test, color='n_counts', spot_size=200)

# Test for extracting the tiles

In [10]:
import os
import torch
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
from scipy.sparse import issparse
import concurrent.futures
import matplotlib.pyplot as plt
import random
import logging
import seaborn as sns
from scipy.spatial import cKDTree  # 用于计算最近邻距离

# 配置 logging 模块，输出详细错误信息
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

def extract_tiles_and_expression(sample, 
                                 tile_size=None, 
                                 tile_size_strategy="median",
                                 tiles_output_dir="output_tiles", 
                                 expr_output_dir="output_expr",
                                 max_workers=32):
    """
    从 HESTSample 对象中提取瓦片和对应的表达量数据，并使用并发线程加速处理。
    
    瓦片大小的自动计算逻辑（基于临近 spot 的距离）：
      - 若 tile_size 为 None，则根据 sample.adata.obs 中每个 spot 的 (pxl_col_in_fullres, pxl_row_in_fullres)
        坐标构建 KD 树，并计算每个点到其最近邻（不包括自身）的欧氏距离。
      - 用户可通过参数 tile_size_strategy 指定采用的统计量：
          * "min"：所有最近邻距离的最小值；
          * "mean"：所有最近邻距离的均值；
          * "median"：所有最近邻距离的中位数（默认）；
          * "max"：所有最近邻距离的最大值。
      - 如果 tile_size 被指定，则直接使用固定值。
    
    瓦片边界处理策略：
      - 对于每个 spot，以其坐标为中心、边长为 tile_size 构造正方形区域。
      - 如果该区域部分在 WSI 范围内，则提取该部分有效区域，并粘贴到完整大小的白色背景上；
      - 如果整个瓦片区域完全不在 WSI 范围内，则认为该点“出界”，不进行处理。
    
    参数:
      sample (HESTSample): 包含空间转录组 AnnData 数据和全分辨率 WSI 的样本实例。
      tile_size (int or None): 瓦片边长（单位像素）。为 None 时自动计算。
      tile_size_strategy (str): 自动计算瓦片大小时采用的策略，可选 "min"、"mean"、"median"、"max"（默认 "median"）。
      tiles_output_dir (str): 保存瓦片图像的目录。
      expr_output_dir (str): 保存表达量数据 (.pt 文件) 的目录。
      max_workers (int): 限制线程池的最大工作线程数，控制并发处理数量，避免内存占用过高。
    
    返回:
      一个列表，每个元素为一个元组：(tile_filename, expr_filename, expr_vector)
      
    函数末尾会随机展示8个在范围内的瓦片及对应 spot 的 top5 基因表达的横向柱状图，
    柱状图使用 seaborn 绘制，其画布与瓦片图像的子图大小一致。
    """
    os.makedirs(tiles_output_dir, exist_ok=True)
    os.makedirs(expr_output_dir, exist_ok=True)
    
    obs_df = sample.adata.obs

    # 1. 检查坐标列是否存在。如果不存在，则使用 adata.obsm['spatial'] 作为坐标来源。
    if 'pxl_col_in_fullres' not in obs_df.columns or 'pxl_row_in_fullres' not in obs_df.columns:
        logging.info("Columns 'pxl_col_in_fullres' and/or 'pxl_row_in_fullres' not found in obs. Using adata.obsm['spatial'] as coordinates.")
        # 假设 adata.obsm['spatial'] 的第一列为 x 坐标，第二列为 y 坐标
        obs_df['pxl_col_in_fullres'] = sample.adata.obsm['spatial'][:, 0]
        obs_df['pxl_row_in_fullres'] = sample.adata.obsm['spatial'][:, 1]

    # 2. 如果 tile_size 为 None，则自动计算
    if tile_size is None:
        points = np.column_stack((obs_df['pxl_col_in_fullres'], obs_df['pxl_row_in_fullres']))
        tree = cKDTree(points)
        dists, _ = tree.query(points, k=2)
        nn_dists = dists[:, 1]  # 排除自身的最近邻距离
        if tile_size_strategy == "min":
            tile_size = int(round(np.min(nn_dists)))
        elif tile_size_strategy == "mean":
            tile_size = int(round(np.mean(nn_dists)))
        elif tile_size_strategy == "max":
            tile_size = int(round(np.max(nn_dists)))
        else:  # 默认 "median"
            tile_size = int(round(np.median(nn_dists)))
        logging.info(f"自动计算瓦片大小：{tile_size} 像素 (策略: {tile_size_strategy})")
    
    X = sample.adata.X
    full_width, full_height = sample.wsi.dimensions

    valid_spots = []
    out_of_bound_spots = []
    for i in range(obs_df.shape[0]):
        spot = obs_df.iloc[i]
        col_coord = int(round(spot['pxl_col_in_fullres']))
        row_coord = int(round(spot['pxl_row_in_fullres']))
        half_tile = tile_size // 2
        top_left_x = col_coord - half_tile
        top_left_y = row_coord - half_tile
        # 判断是否完全出界：右边界 ≤ 0 或 下边界 ≤ 0 或 左边界 ≥ full_width 或 上边界 ≥ full_height
        if (top_left_x + tile_size) <= 0 or (top_left_y + tile_size) <= 0 or \
           top_left_x >= full_width or top_left_y >= full_height:
            out_of_bound_spots.append((i, col_coord, row_coord, top_left_x, top_left_y))
        else:
            valid_spots.append((i, col_coord, row_coord, top_left_x, top_left_y))
    
    logging.info(f"完全出界的点数：{len(out_of_bound_spots)}")
    
    # 随机展示最多 8 个出界的瓦片以供检查
    if out_of_bound_spots:
        sample_indices = random.sample(out_of_bound_spots, min(8, len(out_of_bound_spots)))
        fig, axes = plt.subplots(1, len(sample_indices), figsize=(16, 2))
        if len(sample_indices) == 1:
            axes = [axes]
        for ax, (i, col_coord, row_coord, top_left_x, top_left_y) in zip(axes, sample_indices):
            img = Image.new("RGB", (tile_size, tile_size), (255, 255, 255))
            draw = ImageDraw.Draw(img)
            draw.text((10, tile_size//2 - 10), "Out-of-bound", fill=(255, 0, 0))
            ax.imshow(img)
            ax.axis("off")
            ax.set_title(f"Idx {i}\n({col_coord}, {row_coord})", fontsize=8)
        plt.show()
    
    def process_tile(args):
        i, col_coord, row_coord, top_left_x, top_left_y = args
        # 构造文件名
        tile_filename = os.path.join(tiles_output_dir, f"{sample.sample_id}_{row_coord}_{col_coord}.png")
        expr_filename = os.path.join(expr_output_dir, f"{sample.sample_id}_{row_coord}_{col_coord}.pt")
        
        # 3. 如果文件已存在，则跳过当前 spot 的处理
        if os.path.exists(tile_filename) and os.path.exists(expr_filename):
            logging.info(f"文件已存在，跳过: {tile_filename} 和 {expr_filename}")
            expr_vector = X[i, :]
            if issparse(expr_vector):
                expr_vector = expr_vector.toarray().squeeze()
            else:
                expr_vector = np.array(expr_vector).squeeze()
            return (tile_filename, expr_filename, expr_vector)
        
        left = max(top_left_x, 0)
        top = max(top_left_y, 0)
        right = min(top_left_x + tile_size, full_width)
        bottom = min(top_left_y + tile_size, full_height)
        try:
            region = sample.wsi.read_region((left, top), 0, (right - left, bottom - top))
            region = region.convert("RGB")
        except Exception as e:
            logging.error(
                f"瓦片提取失败，索引 {i}，中心坐标 ({col_coord}, {row_coord})，"
                f"瓦片左上角 ({top_left_x}, {top_left_y})。异常类型：{type(e).__name__}，错误信息：{e}",
                exc_info=True
            )
            return None
        
        tile_img = Image.new("RGB", (tile_size, tile_size), (255, 255, 255))
        offset_x = left - top_left_x
        offset_y = top - top_left_y
        tile_img.paste(region, (offset_x, offset_y))
        
        expr_vector = X[i, :]
        if issparse(expr_vector):
            expr_vector = expr_vector.toarray().squeeze()
        else:
            expr_vector = np.array(expr_vector).squeeze()
        expr_tensor = torch.tensor(expr_vector, dtype=torch.float32)
        expr_sparse = expr_tensor.to_sparse()
        
        try:
            tile_img.save(tile_filename)
        except Exception as e:
            logging.error(
                f"保存瓦片图像失败，索引 {i}，文件名：{tile_filename}。异常：{e}",
                exc_info=True
            )
            return None
        try:
            torch.save(expr_sparse, expr_filename)
        except Exception as e:
            logging.error(
                f"保存表达数据失败，索引 {i}，文件名：{expr_filename}。异常：{e}",
                exc_info=True
            )
            return None
        
        return (tile_filename, expr_filename, expr_vector)
    
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_tile, args) for args in valid_spots]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="处理有效瓦片"):
            result = future.result()
            if result is not None:
                results.append(result)
    
    # 随机展示8个在范围内的瓦片及其 top5 基因表达柱状图（上下两行）
    if results:
        sample_results = random.sample(results, min(8, len(results)))
        n = len(sample_results)
        fig, axes = plt.subplots(2, n, figsize=(n * 3, 2 * 3))
        if n == 1:
            axes = np.array([[axes[0]], [axes[1]]])
        for j, (tile_filename, expr_filename, expr_vector) in enumerate(sample_results):
            # 上行：展示瓦片图像
            try:
                img = Image.open(tile_filename)
                axes[0, j].imshow(img)
                axes[0, j].axis("off")
                title = os.path.basename(tile_filename).replace(".png", "")
                axes[0, j].set_title(title, fontsize=8)
            except Exception as e:
                logging.error(f"展示瓦片失败: {tile_filename}，异常: {e}", exc_info=True)
                axes[0, j].axis("off")
            
            # 下行：展示 top5 基因表达的横向柱状图，使用 seaborn 绘制
            try:
                expr_arr = np.array(expr_vector)
                top5_idx = np.argsort(expr_arr)[-3:][::-1]
                top5_values = expr_arr[top5_idx]
                gene_names = np.array(sample.adata.var_names)[top5_idx]
                sns.barplot(x=top5_values, y=gene_names, ax=axes[1, j], palette="Set2", hue=gene_names)
                axes[1, j].set_xlabel("")
                axes[1, j].set_ylabel("")
                axes[1, j].tick_params(axis='y', labelsize=6)
                axes[1, j].tick_params(axis='x', labelsize=6)
            except Exception as e:
                logging.error(f"展示柱状图失败: {expr_filename}，异常: {e}", exc_info=True)
                axes[1, j].axis("off")
        plt.tight_layout()
        plt.show()
    
    return results

In [11]:
import os
os.chdir("/home1/jijh/diffusion_project/ADiffusion")
from src.pipeline.hest_loading import HESTDataset

In [4]:
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
dataset = HESTDataset(data_dir=data_dir)

# 查询器官为 "Brain" 
brain_samples = dataset.get_samples(
    organ="Brain", 
    oncotree_code=None,  # 根据需要填写
    sample_ids=None,       # 根据需要填写
    species='Mus musculus'          # 根据需要填写
)


print(f"找到 {len(brain_samples)} 个样本。")
print(brain_samples[:3])  # 仅预览前三个

In [5]:
# 随便选一个样本
sample = brain_samples[2]
print(sample)
sample.load_st_data()
sample.load_wsi()

In [33]:
sample.wsi.dimensions

In [34]:
sample.adata.obs

In [6]:
# --- (1) 创建综合可视化图 ---
# 默认使用预生成的空间转录组图像，如果没有则实时计算
sample.visualize_comparison(
    color='clusters',
    use_precomputed_spatial_plot=True  # 可以手动切换到 False 以实时计算
)

In [7]:
tiles_output_dir = '/cwStorage/nodecw_group/jijh/hest_output/output_tiles'
expr_output_dir = '/cwStorage/nodecw_group/jijh/hest_output/output_expr'

extract_tiles_and_expression(sample, tile_size=None, tiles_output_dir=tiles_output_dir, expr_output_dir=expr_output_dir)

In [25]:
tiles_output_dir = '/cwStorage/nodecw_group/jijh/hest_output/output_tiles'
expr_output_dir = '/cwStorage/nodecw_group/jijh/hest_output/output_expr'

extract_tiles_and_expression(sample, tile_size=None, tiles_output_dir=tiles_output_dir, expr_output_dir=expr_output_dir)

In [9]:
save_dir = '/cwStorage/nodecw_group/jijh/hest_output'
all_results = {}
for sample in tqdm(brain_samples, desc="处理 Brain 样本"):
    logging.info(f"开始处理样本 {sample.sample_id} ...")
    sample.load_st_data()
    sample.load_wsi()
    # 为每个样本建立独立的输出文件夹，避免文件混淆
    sample_tiles_dir = os.path.join(save_dir, f"{sample.sample_id}_tiles")
    sample_expr_dir = os.path.join(save_dir, f"{sample.sample_id}_expr")
    
    results = extract_tiles_and_expression(
        sample, 
        tile_size=None, 
        tile_size_strategy="median",
        tiles_output_dir=sample_tiles_dir, 
        expr_output_dir=sample_expr_dir,
        max_workers=32
    )
    all_results[sample.sample_id] = results
    logging.info(f"样本 {sample.sample_id} 处理完成，共处理瓦片数量：{len(results)}")

In [11]:
sample.adata.obs

In [15]:
sample.visualize_comparison(
    color='clusters',
    use_precomputed_spatial_plot=True  # 可以手动切换到 False 以实时计算
)

In [16]:
brain_samples

In [18]:
test_sample = brain_samples[-1]

In [19]:
test_sample.load_st_data()
test_sample.load_wsi()


In [ ]:
test_sample.adata.obs

In [23]:
test_sample.adata.obsm['spatial']

In [21]:
test_sample.visualize_comparison(
    color='clusters',
    use_precomputed_spatial_plot=True  # 可以手动切换到 False 以实时计算
)

In [ ]:
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
dataset = HESTDataset(data_dir=data_dir)

# 查询器官为 "Brain" 
brain_samples = dataset.get_samples(
    organ="Brain", 
    oncotree_code=None,  # 根据需要填写
    sample_ids=None,       # 根据需要填写
    species='Mus musculus'          # 根据需要填写
)


print(f"找到 {len(brain_samples)} 个样本。")
print(brain_samples[:3])  # 仅预览前三个

In [26]:
save_dir = '/cwStorage/nodecw_group/jijh/hest_output'
all_results = {}
for sample in tqdm(brain_samples, desc="处理 Brain 样本"):
    logging.info(f"开始处理样本 {sample.sample_id} ...")
    sample.load_st_data()
    sample.load_wsi()
    # 为每个样本建立独立的输出文件夹，避免文件混淆
    sample_tiles_dir = os.path.join(save_dir, f"{sample.sample_id}_tiles")
    sample_expr_dir = os.path.join(save_dir, f"{sample.sample_id}_expr")
    
    results = extract_tiles_and_expression(
        sample, 
        tile_size=None, 
        tile_size_strategy="median",
        tiles_output_dir=sample_tiles_dir, 
        expr_output_dir=sample_expr_dir,
        max_workers=32
    )
    all_results[sample.sample_id] = results
    logging.info(f"样本 {sample.sample_id} 处理完成，共处理瓦片数量：{len(results)}")

# Process all the samples

In [ ]:
data_dir = "/cwStorage/nodecw_group/jijh/hest_1k"
dataset = HESTDataset(data_dir=data_dir)


all_samples = dataset.get_samples(
    organ=None, 
    oncotree_code=None,  # 根据需要填写
    sample_ids=None,       # 根据需要填写
    species=None        # 根据需要填写
)




In [14]:
print(f"找到 {len(all_samples)} 个样本。")
print(all_samples[:3])  # 仅预览前三个

In [16]:
all_samples

In [15]:
save_dir = '/cwStorage/nodecw_group/jijh/hest_output'
all_results = {}

# Create a list to track failed samples
failed_samples = []

# Use all_samples instead of brain_samples
for sample in tqdm(all_samples, desc="处理所有样本"):
    try:
        # 为每个样本建立独立的输出文件夹
        sample_tiles_dir = os.path.join(save_dir, f"{sample.sample_id}_tiles")
        sample_expr_dir = os.path.join(save_dir, f"{sample.sample_id}_expr")
        
        # Check if the directories already contain processed files
        if os.path.exists(sample_tiles_dir) and os.path.exists(sample_expr_dir):
            # Check if any files exist in these directories
            if len(os.listdir(sample_tiles_dir)) > 0 and len(os.listdir(sample_expr_dir)) > 0:
                logging.info(f"样本 {sample.sample_id} 已经处理过，跳过")
                continue
        
        logging.info(f"开始处理样本 {sample.sample_id} ...")
        sample.load_st_data()
        sample.load_wsi()
        
        results = extract_tiles_and_expression(
            sample, 
            tile_size=None, 
            tile_size_strategy="median",
            tiles_output_dir=sample_tiles_dir, 
            expr_output_dir=sample_expr_dir,
            max_workers=32
        )
        
        all_results[sample.sample_id] = results
        logging.info(f"样本 {sample.sample_id} 处理完成，共处理瓦片数量：{len(results)}")
        
    except Exception as e:
        logging.error(f"处理样本 {sample.sample_id} 时出错: {str(e)}", exc_info=True)
        failed_samples.append((sample.sample_id, str(e)))

# 打印失败的样本列表
if failed_samples:
    logging.warning(f"处理失败的样本数量: {len(failed_samples)}")
    for sample_id, error in failed_samples:
        logging.warning(f"  - {sample_id}: {error}")
else:
    logging.info("所有样本处理成功！")